### Import Libraries

In [ ]:
import os
import json
from tqdm import tqdm
import requests
from collections import Counter
import seaborn as sns
import pandas as pd
#import dataframe_image as dfi
import numpy as np
import matplotlib.pyplot as plt
#import matplotlib.patches as mpatches

from datetime import date, datetime as dt
import time,datetime
import re
import unicodedata
import ast  # Module to handle literal_eval function

In [ ]:
# Targeting the retraction_index_path
retraction_index_path = os.path.abspath('./retraction-indexing-agreement/')
retraction_index_path

data_dir = retraction_index_path+'/dataset/' # data directory
result_dir = retraction_index_path +'/result/'


#### Configuration File

In [ ]:
# Load configuration
con_file = open(retraction_index_path+"\config.json")
config = json.load(con_file)
con_file.close()

# Initializing variable for configuration file
my_email = config['my_email']
elsevier_api_key = config['Elsevier_APIKEY']
elsevier_insttoken = config['insttoken']
ieee_xplore_api_key = config['IEEEXplore_APIKEY']
wos_api_key = config['WoS_APIKEY']

In [ ]:
# Global initializatiion
global my_email
global elsevier_api_key
global elsevier_insttoken
global ieee_xplore_api_key
global wos_api_key

In [ ]:
os.listdir(data_dir)

### Creating Present Union list As At July2024

In [ ]:
"""
Input the date you retrieve retraction publications for each database

Update the date for each database format: YYYY-MM-DD e.g. 2024-02-13
"""

getdate = {'scopus': '2024-07-05',
            'crossref':'2024-07-03',
            'retractionwatch': '2024-07-03',
            'webofsciencecore': '2024-07-03'} 


In [ ]:
def convert_unicode(string: str) -> str:
    """
    It takes a string and passes it through different encoding parameter phases
    E.g. '10.\u200b1105/\u200btpc.\u200b010357' ->  '10.1105/tpc.010357'
    
    :param string: variable to be encoded
    :return: the actual string value devoided of encoded character
    """
    
    string = unicodedata.normalize('NFKD', string).encode('iso-8859-1', 'ignore').decode('iso-8859-1')
    string = unicodedata.normalize('NFKD', string).encode('latin1', 'ignore').decode('latin1')
    string = unicodedata.normalize('NFKD', string).encode('cp1252', 'ignore').decode('cp1252')
    return string

In [ ]:
"""
- Reading in Crossref file and renaming some columns
"""
crossref = pd.read_csv(data_dir+'crossref/crossref_retractedpublication_'+\
                     getdate['crossref']+'.csv').drop('Unnamed: 0',axis=1)
crossref['source'] = 'Crossref'

crossref['DOI']= crossref['DOI'].str.lower().str.strip().astype(str).apply(convert_unicode)
crossref.head(2)

In [ ]:
"""
- Reading in Retraction Watch items
"""

retractionwatch = pd.read_csv(data_dir+'/retractionwatch/retractionwatch_'+\
                     getdate['retractionwatch']+'.csv', encoding='latin1').rename(
    columns={'OriginalPaperDOI':'DOI', 
             'OriginalPaperPubMedID': 'PubMedID', 
             'OriginalPaperDate': 'Year'})

retractionwatch['source']='Retraction Watch'


retractionwatch['PubMedID']= retractionwatch['PubMedID'].fillna(0).astype(int)\
                .replace(0,'').astype(str).str.strip()

retractionwatch['RetractionPubMedID']= retractionwatch['RetractionPubMedID'].fillna(0).astype(int)\
                .replace(0,'').astype(str)

retractionwatch['Year']=  pd.to_datetime(retractionwatch['Year'], exact=False).dt.year
retractionwatch['RetractionYear']=  pd.to_datetime(retractionwatch['RetractionDate'], exact=False).dt.strftime("%Y").fillna(0).astype(int)

# A row in retraction Watch contains '|' from a row in 
retractionwatch['DOI']= retractionwatch['DOI'].str.replace(r'\|', '') # "10.1038/embor.2009.88 |"

retractionwatch['DOI'] = retractionwatch['DOI'].str.lower().fillna('').str.strip().apply(convert_unicode)

retractionwatch.head()

In [ ]:
"""
- Reading in Scopus file and renaming some columns
"""
scopus = pd.read_csv(data_dir+'scopus/scopus_retractedpublication_'+\
                     getdate['scopus']+'.csv').rename(
    columns={'Authors':'Author',
             'Source':'source',
             'Titles':'Title',
             'Source title':'Journal',
             'PubMed ID': 'PubMedID'})



# Converting 'Publication_date' to Date to extract the year
scopus['Publication_date'] = pd.to_datetime(scopus['Publication_date'])
scopus['Year'] = scopus['Publication_date'].dt.year

scopus['DOI']= scopus['DOI'].str.lower().str.strip().astype(str).apply(convert_unicode)

scopus['PubMedID']= scopus['PubMedID'].fillna(0).astype(int)\
                .replace(0,'').astype(str).str.strip()

scopus['source'] = 'Scopus'

scopus.head(2)

In [ ]:
"""
Loading in indexed retracted publications from Web of Science Core
"""

webofsciencecore= pd.read_csv(data_dir+f'webofscience/webofsciencecore_retractedpublication_'+\
                     getdate['webofsciencecore']+'.csv')

webofsciencecore['source'] = 'WoS_Core'

webofsciencecore['DOI']= webofsciencecore['DOI'].str.lower().str.strip().astype(str).apply(convert_unicode)

# Remove decimal part
webofsciencecore['PubMedID'] = webofsciencecore['PubMedID'].fillna('').astype(str).str.split('.').str[0]

webofsciencecore.head(2)

In [ ]:
def checkindividualdataset(x)
    """
    :param x: dataframe object
    Clean and deplicate records based on DOIs.
    After removing duplicates, we will return the count and the list of records with DOI, 
    those without DOIs and duplicated records that will be dropped.
    """
    
    # Getting the DataFrame name
    df_name = [name for name, obj in globals().items() if obj is x][0]

    
    # Step 1: We identify the unique records of each dataset based on DOI.
    # 'records_withDOI_hasdup': Identify records that have a valid DOI which should start with '10.'
    # 'records_withDOI': Drop the duplicates from the previous line.
    
    records_withDOI_hasdup= x.loc[x['DOI'].str.startswith('10.', na=False)]
    
    if df_name == 'pubmed':
        records_withDOI = records_withDOI_hasdup.drop_duplicates(subset=['DOI'], keep='last')
    else:
        records_withDOI = records_withDOI_hasdup.drop_duplicates(subset=['DOI'], keep='first')

    
    # Step 2: We create two duplicate lists.
    # 'duplicated_records_all': Identify ALL duplicated records for reference and download for checking manually.
    # 'duplicated_records': Identify duplicated records to drop but keep only the first occurrence of each group of duplicates.
    duplicate_records_all = records_withDOI_hasdup.loc[records_withDOI_hasdup.duplicated(subset=['DOI'],keep=False), :]
    
    
    # To check anamolies - to remove record with PubMedID 28202934 and leave that 26511294 with correct date
    if df_name == 'pubmed':
        duplicate_records = records_withDOI_hasdup.loc[records_withDOI_hasdup.duplicated(subset=['DOI'],keep='last'), :]
    else:
        duplicate_records = records_withDOI_hasdup.loc[records_withDOI_hasdup.duplicated(subset=['DOI'],keep='first'), :]


    
    # Step 3: We get the count of records without DOI. Duplicates may exist since we could not use DOI to identify duplicate records
    records_withoutDOI = x.loc[~x['DOI'].str.startswith('10.', na=False)]    

    
    try:
        # Return the count and items of each group
        if len(x) == len(records_withDOI)+len(records_withoutDOI)+len(duplicate_records):
            return[len(x), len(records_withDOI),len(records_withoutDOI),len(duplicate_records), records_withDOI, records_withoutDOI, duplicate_records, duplicate_records_all]
            
    
    except: 
        return('ERROR')

In [ ]:
def count_DOI_n_PubMedID(df, source)-> list:
    """
    :param df: DataFrame to work on
    :param source: source to look up to determine number of count
    
    :return: source, # DOI, # PubMedID, # Duplicated record -> list
    
    """
    
    df_DOI= df[(df['DOI'].str.startswith('10')) & (df['source'].str.contains(source))]
    
    df_nodupDOI= df_DOI.drop_duplicates(subset=['DOI'], keep='first') # DF that has no duplicated DOI
    
    df_duplicatedDOI= df_DOI[df_DOI.duplicated(subset=['DOI'],keep='last')] # DF that are duplicated
    
    
    df_noDOI= df[(~df['DOI'].str.startswith('10')) & (df['source'].str.contains(source))] # DF that has no DOI
    
    
    nDOI= len(df_nodupDOI) # Numbers of items with unique DOI
    nDuplicatedDOI= len(df_duplicatedDOI) # Numbers of items that has duplicated DOI removed
    nNoDOI= len(df_noDOI) # Numbers of items without DOI
    
    
    
    if 'PubMedID' in df.columns:
        
        df_PMID= df_DOI[((df_DOI['PubMedID'] != "") | ~df_DOI['PubMedID'].isna()) & (df_DOI['source'].str.contains(source))]
        
        df_nodupPMID= df_PMID.drop_duplicates(subset=['DOI'], keep='first') # DF that has no duplicated PMID
        
        df_duplicatedPMID= df_PMID[df_PMID.duplicated(subset=['DOI'],keep='last')] # DF that has duplicated PMID
        
        df_noPMID= df[~(((df['PubMedID'] != "") | ~df['PubMedID'].isna()) & (df['source'].str.contains(source)))]
        
        
        nPMID= len(df_nodupPMID) # Numbers of items with unique PMID
        nDuplicatedPMID= len(df_duplicatedPMID)  # Numbers of items with duplicated PMID
        nNoPMID= len(df_noPMID)  # Numbers of items with without PMID
        
    else:
        nPMID,nDuplicatedPMID,nNoPMID= 0,0,0
        
        
    Total= len(df)
        
    return  source, Total, nDOI, nNoDOI, nDuplicatedDOI, nPMID, nNoPMID, nDuplicatedPMID 

In [ ]:
"""
Counting # DOIs, PubMedID
#  return: source,Total, nDOI,nNoDOI,nDuplicatedDOI,nPMID,nNoPMID,nDuplicatedPMID 
"""
nCrossref= count_DOI_n_PubMedID(crossref,'Crossref')
nRW= count_DOI_n_PubMedID(retractionwatch,'Retraction Watch')
nScopus= count_DOI_n_PubMedID(scopus,'Scopus')
nWoS=count_DOI_n_PubMedID(webofsciencecore,'WoS_Core')

In [ ]:
print(nCrossref)

In [ ]:
print(nRW)

In [ ]:
print(nScopus)

In [ ]:
print(nWoS)

In [ ]:
"""
Aggregate the items from all the sources
"""
dbtable = [] # A nested list which stores the records of each group in each source
ovtable = [] # Store the count of each group from each source and create a table for viewing

dblist= [nCrossref, nRW, nScopus, nWoS]

# Query results retrieved	Records with DOI	Records without DOI removed	Duplicate records removed
# source,Total, nDOI,nNoDOI,nDuplicatedDOI,nPMID,nNoPMID,nDuplicatedPMID
for result in dblist:
    dbtable.append(result)
    
    np_results= np.array(dbtable)
    
# Create a table showing the count of each group
overview = pd.DataFrame(np_results[:,[1,2,3,4,5]])
overview.columns =['Query_result', 'Records_withDOI', 'Records_withoutDOI', 'Duplicate_DOI_removed', 'DOI_records_withPubMedID']
overview['source']= ['Crossref','Retraction Watch','Scopus','Web of Science Core']

# Re-order column
overview = overview[['source', 'Query_result', 'Records_withDOI', 'Records_withoutDOI', 'Duplicate_DOI_removed', 'DOI_records_withPubMedID']]

# Aggregating items in each column
overview.loc[len(overview)] = ['Total',overview.Query_result.astype(int).sum(), 
                               overview.Records_withDOI.astype(int).sum(),
                               overview.Records_withoutDOI.astype(int).sum(), 
                               overview.Duplicate_DOI_removed.astype(int).sum(), 
                               overview.DOI_records_withPubMedID.astype(int).sum(),] 



overview\
#         .to_csv(result_dir+'datasources_overview_unionlist_crws.csv')

###### Exporting Records to Folder
- These include DOIs, deduplicated DOIs, and No DOI records

In [ ]:
def export_datasets_from_sources(source_list: list, dbtable: list):
    """
    This function filter records with doi,without doi, and those with duplicated doi
    for each source and save each file
    
    :param source_list: list of dataframe of sources. This is dblist. It is initiated later
    :return: It will for each source return: 
            - records with doi (saved as in this format: 'source_name_recordswithdoi_date')
            - records without doi (saved as in this format: 'source_name_recordsnodoi_date')
            - duplicated doi records (saved as in this format: 'source_name_duplicatedrecords_date')
    """
    
    
    for i in range(len(source_list[:-1])):
        source_name = [name for name, obj in globals().items() if obj is source_list[i]][0]

        date = getdate[source_name]
        
        print(f'{source_name} was last updated on {date}')
        
        if source_name in ['webofsciencecore']:
            dbtable[i][4].sort_values(by=['DOI'], ascending=False).to_csv(data_dir+ 'webofscience'+'/'+source_name+'_recordswithdoi_'+date + '.csv')
            dbtable[i][5].to_csv(data_dir+ 'webofscience'+'/'+source_name+'_recordsnodoi_' + date + '.csv')
            dbtable[i][6].sort_values(by=['DOI'], ascending=False).to_csv(data_dir+ 'webofscience'+'/'+source_name+ '_duplicatedrecords_' + date + '.csv')

      
        else:
            dbtable[i][4].sort_values(by=['DOI'], ascending=False).to_csv(data_dir+source_name+'/'+source_name+'_recordswithdoi_'+date + '.csv')
            dbtable[i][5].to_csv(data_dir+ source_name+'/'+source_name+'_recordsnodoi_' + date + '.csv')
            dbtable[i][6].sort_values(by=['DOI'], ascending=False).to_csv(data_dir+ source_name+'/'+source_name+ '_duplicatedrecords_' + date + '.csv')


      

In [ ]:
# Saving records of items with DOIs, without DOIs, and duplicated records in each of the sources

dbtable2 = [] # A nested list which stores the records of each group in each source
dblist2 = [crossref,retractionwatch, scopus, webofsciencecore,'']

for x in dblist2[:-1]:
    dbtable2.append(checkindividualdataset(x))

export_datasets_from_sources(dblist2[:], dbtable2)

In [ ]:
"""
Getting Records without DOI
"""
for record in dbtable:
    source = record[0]
    if record[0]=='WoS_Core':
        source = 'Web of Science'
    print(f"The total # records without DOI in {source} is {record[3]}")

In [ ]:
"""
Loading records lacking DOIs
"""

nodoi_crossref_df= pd.read_csv(data_dir+'crossref/crossref_recordsnodoi_'+ getdate['crossref'] + '.csv')
nodoi_scopus_df= pd.read_csv(data_dir+'scopus/scopus_recordsnodoi_'+ getdate['scopus'] + '.csv')
nodoi_rw_df= pd.read_csv(data_dir+'retractionwatch/retractionwatch_recordsnodoi_'+ getdate['retractionwatch'] + '.csv')
nodoi_wos_df= pd.read_csv(data_dir+'webofscience/webofsciencecore_recordsnodoi_'+ getdate['webofsciencecore'] + '.csv')

In [ ]:
"""
Calculating Records with PMIDs lacking DOIs in the newly-added records

pmid_nodoi_srw  <- is the PMIDs of records that lack DOIs in union list of Retraction Watch, Web of Sciece, and Scopus. Crossref always has DOI.
sti_nodoi_pmids_filtered <- is the PMIDs of records that lack DOIs in STI2023 union list 
"""

nodoi_scopus_df['PubMedID']= nodoi_scopus_df['PubMedID'].fillna(0).astype(int).astype(str)
nodoi_rw_df['PubMedID']= nodoi_rw_df['PubMedID'].fillna(0).astype(int).astype(str)
nodoi_wos_df['PubMedID']= nodoi_wos_df['PubMedID'].fillna(0).astype(int).astype(str)


pmid_nodoi_srw =set(nodoi_scopus_df[nodoi_scopus_df['PubMedID']!='0']['PubMedID'].tolist()+\
                    nodoi_rw_df[nodoi_rw_df['PubMedID']!='0']['PubMedID'].tolist() +\
                    nodoi_wos_df[nodoi_wos_df['PubMedID']!='0']['PubMedID'].tolist())

print(f"The total # PMIDs of records lacking DOIs is {len(pmid_nodoi_srw)} in the union list ")

In [ ]:
"""
Laoding retracted records with DOI
"""
crossref_retracted= pd.read_csv(data_dir+'crossref/crossref_recordswithdoi_'+getdate['crossref'] +'.csv')

retractionwatch_retracted= pd.read_csv(data_dir+'retractionwatch/retractionwatch_recordswithdoi_'+getdate['retractionwatch'] +'.csv')

scopus_retracted = pd.read_csv(data_dir+'scopus/scopus_recordswithdoi_'+getdate['scopus']+'.csv')

webofsciencecore_retracted = pd.read_csv(data_dir+'webofscience/webofsciencecore_recordswithdoi_'+getdate['webofsciencecore'] +'.csv')


In [ ]:
# Merged into one full list 'The Union List'

"""
Select PubMed with doi and 
Select the columns: 'DOI', 'Author','Title', 'Year', 'Journal', 'source', 'PubMedID' -> 'union_list'
"""
retracted_sources= [crossref_retracted, retractionwatch_retracted, scopus_retracted, webofsciencecore_retracted]

merged_withdoi = pd.concat(retracted_sources)
merged_withdoi = merged_withdoi[['DOI', 'Author','Title', 'Year', 'Journal', 'source', 'PubMedID']].sort_values(by='DOI')

# Check if the number of records are consistent before and after merging.
if len(merged_withdoi) == (len(crossref_retracted) +len(retractionwatch_retracted)+len(scopus_retracted)+ len(webofsciencecore_retracted)):
    print('full record count:', len(merged_withdoi))
else:
    print('ERROR: Inconsistent Counts')

In [ ]:
merged_withdoi['source']=merged_withdoi['source'].astype(str)

In [ ]:
"""
It merges file from all the sources using their DOIs, and joins the source that of the items
Output File: One merged CSV file as final output - 

Uses date as of today (last update): '2023-07-09 to indicate last update the items were merged

"""

unionlist_crws = merged_withdoi.groupby('DOI').agg({'Author':'first', 
                              'Title': 'last',
                              'Year': 'first',
                              'Journal': 'last',
                              'source':'; '.join, 
                              'PubMedID':'first'}).reset_index()

unionlist_crws['PubMedID']= unionlist_crws['PubMedID'].fillna(0).astype(int).replace(0,'').astype(str)
unionlist_crws

In [ ]:
"""
Get number of unique PMID
"""
print(f"The total # of unique PMID in the union list is {unionlist_crws['PubMedID'].nunique()}")

#### Loading coverage items not indexed for each source

In [ ]:
"""
Input the date you retrieve retraction publications for each database

Update the date for each database format: YYYY-MM-DD e.g. 2024-02-13
"""

date_coverage = {'scopus': '2024-08-02',
            'crossref':'2024-08-06',
            'webofsciencecore': '2024-07-30'} 

In [ ]:
"""
Get coverage for the sources - -
Read in items that were covered but not indexed as retracted publication from each sources <- _notindexed
"""

crossref_notindexed= pd.read_csv(data_dir+'coverednotindexed/crossref_coverednotindexed_'+date_coverage['crossref']+'.csv').drop('Unnamed: 0',axis=1)
crossref_notindexed['source']='Crossref'

scopus_notindexed= pd.read_csv(data_dir+'coverednotindexed/scopus_coverednotindexed_'+date_coverage['scopus']+'.csv').drop('Unnamed: 0',axis=1)
scopus_notindexed['source']='Scopus'

woscore_notindexed= pd.read_csv(data_dir+'coverednotindexed/webofsciencecore_coverednotindexed_'+date_coverage['webofsciencecore']+'.csv')
woscore_notindexed['source']='WoS_Core'

In [ ]:
unionlist_indexed= unionlist_crws.copy()

In [ ]:
"""
Merge all the items covered but not indexed as retracted publication from each source with the union list
<- unionlist_covers
"""

merged_withdoi2 = pd.concat([unionlist_indexed, crossref_notindexed,scopus_notindexed,
                           woscore_notindexed,])

unionlist_covers= merged_withdoi2.groupby('DOI').agg({'Author':'first', 
                              'Title': 'first',
                              'Year': 'first', 
                              'Journal': 'first',
                              'PubMedID':'first',
                              'source':'; '.join 
                              }).reset_index()

unionlist_covers.fillna('', inplace=True)


In [ ]:
unionlist_covers

In [ ]:
"""
Deduplicate the 'source' in the unionlist_covers (i.e. that entails covered all items and 
call the source field -> 'source_new'.

"""

# Deduplicating source value 
unionlist_covers['source'] = unionlist_covers['source'].apply(lambda x: '; '.join(sorted(set(x.split('; ')))))

# Renaming column
unionlist_covers.rename(columns={'source': 'source_new'}, inplace=True)
unionlist_covers

In [ ]:
"""
Merge unionlist_covers (that details covers items) and unionlist_indexed (that detailed indexed retracted items)
Call source in unionlist_indexed -> 'source_old'
"""

unionlist_crws_covered_and_indexed = pd.merge(unionlist_covers, unionlist_indexed[['DOI', 'source', ]].rename(columns={'source': 'source_old'}), on='DOI')

# Store 'source_old' & 'source_new' value as sorted list
unionlist_crws_covered_and_indexed['source_old'] = unionlist_crws_covered_and_indexed['source_old'].apply(lambda x: sorted(x.split('; ')))#.astype(str)
unionlist_crws_covered_and_indexed['source_new'] = unionlist_crws_covered_and_indexed['source_new'].apply(lambda x: sorted(x.split('; ')))#.astype(str)

unionlist_crws_covered_and_indexed['Year']= unionlist_crws_covered_and_indexed['Year'].astype(int)

unionlist_crws_covered_and_indexed.head()

#### Calculating RetractionIndexingAgreement Source

In [ ]:
"""
Calculating Retraction Indexing Agreement
"""
count_sources_covering_item = unionlist_crws_covered_and_indexed.source_new.apply(len)

count_sources_indexing_item_as_retracted = unionlist_crws_covered_and_indexed.source_old.apply(len)#str.len()
RetractionIndexingDiscrepancy_ITEM = count_sources_indexing_item_as_retracted / count_sources_covering_item

# add and show the calculation score to dataframe
unionlist_crws_covered_and_indexed['RetractionIndexingAgreement_ITEM(%)'] = ((RetractionIndexingDiscrepancy_ITEM) * 100).astype(int)

unionlist_crws_covered_and_indexed

In [ ]:
unionlist_crws_covered_and_indexed.to_csv(result_dir + 'unionlist_crws_ria_2024-07-09.csv')

In [ ]:
def count_source(df: pd.DataFrame, source_name: str)-> tuple:
    """
    :param df: DataFrame to work on
    :param source_name: source to lookup to determine number of count
    
    :return: source, # no_inOLD, no_inNEW -> list
    
    """
    
    no_inOLD= len(set(df[(df['source_old'].apply(lambda x: source_name in x))]['DOI'])) #.count()[0]
    
    no_inNEW= len(set(df[(df['source_new'].apply(lambda x: source_name in x))]['DOI'])) #.count()[0]

    
    return  no_inOLD,no_inNEW

In [ ]:
"""
Counting # items indexed and covered in each sources
"""

crossref_count= count_source(unionlist_crws_covered_and_indexed, 'Crossref')

scopus_count= count_source(unionlist_crws_covered_and_indexed, 'Scopus')

webofsciencecore_count= count_source(unionlist_crws_covered_and_indexed, 'WoS_Core')



In [ ]:
"""
Total Indexed & Coverage
"""
print(f" In Crossref, the total indexed items is {crossref_count[0]} and the total coverage is {crossref_count[1]}")
print(f" In Scopus, the total indexed items is {scopus_count[0]} and the total coverage is {scopus_count[1]}")
print(f" In Web of Science, the total indexed items is {webofsciencecore_count[0]} and the total coverage is {webofsciencecore_count[1]}")

In [ ]:
"""
Calculating # items not indexed as retracted for each source -> not_covered
"""
not_covered= [len(unionlist_crws_covered_and_indexed) - crossref_count[1],
              len(unionlist_crws_covered_and_indexed) - scopus_count[1],
              len(unionlist_crws_covered_and_indexed) - webofsciencecore_count[1]]

In [ ]:
"""
- Computing the Retraction Indexing Agreement score for sources (indexed sources only)
- Putting results of # indexed, coverednotindexed, not_covered
"""

table = pd.DataFrame()
table['source']= ['Crossref', 'Scopus', 'Web of Science Core']
table['indexed_as_retracted']= [crossref_count[0],scopus_count[0],webofsciencecore_count[0]]

table['covered_but_not_indexed_as_retracted'] = [crossref_count[1]-crossref_count[0],
                                                 scopus_count[1]-scopus_count[0],
                                                 webofsciencecore_count[1]-webofsciencecore_count[0]]

table['not_covered'] = not_covered

ria_source=[]

for i in range(0,3):
    D = table['indexed_as_retracted'][i] + table['covered_but_not_indexed_as_retracted'][i]   
    ria_source.append(round(((table['indexed_as_retracted'][i]/D)*100), 2))

table['RetractionIndexingAgreement_SOURCE(%)'] = [ria_source[0], ria_source[1], ria_source[2]]
 #set first column as index
table = table.sort_values(by='source')
table = table.reset_index()
table =table.drop('index',axis=1)
table\
    #.to_csv(result_dir+'ria_scores_unionlist_crws.csv')


In [ ]:
overview\
#         .to_csv(result_dir+'datasources_overview_unionlist_crws.csv')

In [ ]:
"""
Load STI2023 union list
"""
unionlist_sti = pd.read_csv(data_dir+"/sti2023/2023-09-03_journalcategory_knownretractionlist_updated.csv").drop(['Unnamed: 0'],axis=1)
unionlist_sti['DOI']= unionlist_sti['DOI'].str.lower()
unionlist_sti

In [ ]:
"""
Loading PMIDs of records with no DOI from STI: Crossref (0), Web of Science (276), Retraction Watch (682), Scopus (8): 966

sti_nodoi_pmids <- publication records with noDOI from dataset that was used for STI2023 union list
"""

with open(data_dir+"sti2023/nodoi_sti_pmids.txt",'r') as fn:#.drop(['Unnamed: 0'],axis=1)
    sti_nodoi_pmids = fn.read().split('\n')
    sti_nodoi_pmids.remove('')
    
print(f'The total unique # of records lacking DOIs but having PMIDs is {len(sti_nodoi_pmids)}')

In [ ]:
"""
Filtering PMIDs of NoDOI items for STI2023 not in STI2023 Union list - 
to remove pre-existing PMID from STI2023 union list
"""
unionlist_sti['PubMedID']=unionlist_sti['PubMedID'].fillna('0').astype(int).astype(str)

# Getting PMIDs in STI2023
sti_nodoi_pmids_filtered = set(sti_nodoi_pmids) - set(unionlist_sti['PubMedID'].tolist())
print(f'Actual records with noDOI with PMID for STI2023 formation:  {len(sti_nodoi_pmids_filtered )}')

In [ ]:
"""
Records with NoDOI but having PMIDs [from Web of Science (276), Retraction Watch (682), Scopus (8)]
for STI2023 unionlist in unionlist_crws
"""

nodoi_sti_found_df= unionlist_crws_covered_and_indexed[(unionlist_crws_covered_and_indexed['PubMedID'] != '') & \
                                                       (unionlist_crws_covered_and_indexed['PubMedID'].isin(sti_nodoi_pmids_filtered))]

print(f'Of 657 NoDOI_withPMID records used for creating STI2023 unionlist, \
only {len(nodoi_sti_found_df)} items were found in Unionlist_CRWS')


In [ ]:
"""
Breakdown analysis in Year 2023 & 2024 in STI2023 & Unionlist_CRWS
"""
n_sti2023 = unionlist_sti[unionlist_sti['Year']==2023].count().iloc[0]
n_ul_crws2023 = unionlist_crws_covered_and_indexed[unionlist_crws_covered_and_indexed['Year'] == 2023].count().iloc[0]
n_ul_crws2024 = unionlist_crws_covered_and_indexed[unionlist_crws_covered_and_indexed['Year'] == 2024].count().iloc[0]


print(f'The # of DOIs indexed as retracted in STI2023 union list in 2023 (Jan - April): {n_sti2023}')
print(f'The # of DOIs indexed as retracted in unionlist_crws in 2023 (Jan - Dec): {n_ul_crws2023}')
print(f'The # of DOIs indexed as retracted in unionlist_crws in 2024 (Jan - July): {n_ul_crws2024}')

In [ ]:
"""
Comparing Total # retracted item in Crossref, Scopus, Retraction Watch, Web of Science Core compared with STI2023
Newly added DOIs from STI2023 to unionlist_crws 2023
"""

newly_added_crws = unionlist_crws_covered_and_indexed[~unionlist_crws_covered_and_indexed['DOI'].isin(unionlist_sti['DOI'])].copy()
newly_added_crws

In [ ]:
"""
Finding DOIs in union list 2024 and not in STI union list: Actual newly added DOIs in Unionlist_crws
"""
diff_notin_sti_doi= set(unionlist_crws_covered_and_indexed['DOI']) - set(unionlist_sti['DOI'])
print(f"The total DOIs in union list CRWS 2024 and not in STI2023 is: {len(diff_notin_sti_doi)}")
print(f"{(len(diff_notin_sti_doi)/len(unionlist_sti))*100}%")

In [ ]:
new_added_doi_withpmid = newly_added_crws[newly_added_crws['PubMedID']!='']
new_added_doi_withpmid

In [ ]:
"""
Data Quality Check:

Common records in STI2023 and in unionlist_crws
"""

common_df= pd.merge(unionlist_crws, unionlist_sti.iloc[:,0], on='DOI', how='inner')
common_df

In [ ]:
"""
Getting items with PMID in the newly-added_crws
"""
new_added_doi_withpmid = newly_added_crws[newly_added_crws['PubMedID']!='']
print(f'# Newly-added items with PMIDs are {len(new_added_doi_withpmid)}')

In [ ]:
"""
Get items retracted that are older year than Year 2023 in the newly_added_crws items
"""

print(f"Total # newly indexed items in newly_added_crws before Year 2023: {(newly_added_crws[newly_added_crws['Year']<2023].count().iloc[0])}")


In [ ]:
"""
Calculating DOIs for Sankey Diagram
"""

print(f"Year 2024:")
n_newly_added_in2024 = newly_added_crws[newly_added_crws['Year']>2023].count()[0]
print(f"Items added in Jan to July 2024 is {n_newly_added_in2024}")

n_doi_newly_added_in2023 = len(newly_added_crws[newly_added_crws['Year']==2023]['DOI'].tolist())
n_doi_in2023_sti= len(unionlist_sti[unionlist_sti['Year']==2023]['DOI'].tolist())

print(f"\nYear 2023:")
print(f"Items added in Jan to Dec 2023 {n_doi_newly_added_in2023}")
print(f"Items added in Jan to April 2023 is {n_doi_in2023_sti}")
print(f"Items added in May to Dec 2023 is {n_doi_newly_added_in2023 - n_doi_in2023_sti}")

# 427 - missing DOIs
print(f"Items remaining in STI2023 from Year1940 -Year2022) is \
{len(unionlist_sti) - 427 -  n_doi_in2023_sti}")

# 11 items from NoDOI but PMID in STI2023; 
print(f"\nOther remaining items in present unionlist CRWS: \
 {len(unionlist_crws_covered_and_indexed) - n_newly_added_in2024 - n_doi_newly_added_in2023 - len(common_df) - 11}")

In [ ]:
len(unionlist_crws_covered_and_indexed)

In [ ]:
"""
Calculating the breakdown of found DOIs in unionlist_crws by sources
"""
print(f'The total # of records with noDOI of STI2023 now in  unionlist_crws 2024 is {len(nodoi_sti_found_df)}')

nodoi_sti_found_source=  [source for source in nodoi_sti_found_df['source_old'].tolist()]
source_list=[]
for sources in nodoi_sti_found_source:
    sources = [source.strip() for source in sources]
    source_list.extend(sources)

print(f'The breakdown of sources those missing DOIs found are:')
Counter(source_list)

In [ ]:
"""
For STI2023
DOI in 2023 (Jan - April): 375
Missing DOI in STI: 425 
Left DOIs between start - Year2022:
"""
len(unionlist_sti) - 425 - 375

In [ ]:
# Calculation common DOIs between STI2023(<2023) and Unionlist_CRWS
sti_till2023= unionlist_sti[unionlist_sti['Year']!=2023]['DOI']
common_doi_between_sti_and_ul_crws_before_2023= set(sti_till2023) & set(unionlist_crws['DOI'])
len(common_doi_between_sti_and_ul_crws_before_2023)

In [ ]:
# Calculating remaining new DOIs for Unionlist for Sankey diagram
len(unionlist_crws) - len(common_df) - 27 - 2603 - 12369

In [ ]:
def count_DOIs_for_RIA(df:pd.DataFrame)-> list():
    n_25= df[df['RetractionIndexingAgreement_ITEM(%)']<=25].count()[0]
    n_to50= df[(df['RetractionIndexingAgreement_ITEM(%)']>25) & \
               (df['RetractionIndexingAgreement_ITEM(%)']<=50)].count()[0]
    
    # For RIA 66 & 75
    n_to75= df[(df['RetractionIndexingAgreement_ITEM(%)']>50) & \
               (df['RetractionIndexingAgreement_ITEM(%)']<=75)].count()[0]
   
    # For RIA at 100
    n_100= df[df['RetractionIndexingAgreement_ITEM(%)']==100].count()[0]
    
    n_count= [n_25, n_to50,n_to75,n_100]
    
    n_frac= []
    for cnt in n_count:
        res= round(cnt/(len(df))*100,2)
        n_frac.append(res)
    
    table=pd.DataFrame()
    table['noDOIs']= n_count
    table['Fraction_in_Unionlist(%)'] = n_frac

    table['RIA_Score(%)'] = ["<=25%",'>25% to 50%','>50% to 75%','100%']
    table['Color']= ['red','orange','#1f77b4','green',]
    
    return table

In [ ]:
"""
# Filtered exclusively indexed sources - items that indexed and covered in only one source
"""
unionlist_crws_filtered= unionlist_crws_covered_and_indexed[~unionlist_crws_covered_and_indexed['source_new'].apply(lambda x: len(x) == 1)].copy()

all_excluded= len(unionlist_crws_covered_and_indexed[unionlist_crws_covered_and_indexed['source_new'].apply(lambda x: len(x) == 1)])

"""
Calculating # items exclusive-indexed  & covered in STI2023 & newly added union list 
"""
print(f"# of exclusively-indexed items excluded in the unionlist is {all_excluded}")

In [ ]:
"""
Distribution of RetractionIndexingAgreement of the entire Unionlist_CRW (exclusive source filtered)
"""
table000 = count_DOIs_for_RIA(unionlist_crws_filtered)
table000

In [ ]:
"""
Retraction Indexing Agreement by Items (in Group) for the Entire Unionlist_CRWS
"""

plt.style.use('bmh')
fig000, ax000 = plt.subplots(figsize=(14, 8))

mycolor_000 = dict(zip(table000['RIA_Score(%)'], table000['Color']))

# Create the bar plot
sns.barplot(y=table000['Fraction_in_Unionlist(%)'], x=table000['RIA_Score(%)'],
            ax=ax000, color="#1f77b4",palette=mycolor_000, ci=None)

#Add custom legend
legend_labels = {
    'red': '<=25%',
    'orange': '>25% to 50%',
    '#1f77b4': '>50% to 75%',
    'green': '100%'
}

legend_handles = [mpatches.Patch(color=color, label=label) for color, label in legend_labels.items()]

ax000.legend(handles=legend_handles, title="RIA Score Range", loc="upper left")


# Label bars with noDOIs values
for bar, noDOIs, frc in zip(ax000.patches, table000['noDOIs'],table000['Fraction_in_Unionlist(%)']):
    ax000.annotate(f"{noDOIs} Items ({frc}%)", #f"{noDOIs} / {len(unionlist3)}"
                      xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
                      xytext=(0, 3),  # 3 points vertical offset
                      textcoords="offset points",
                      ha='center', va='bottom')

# Set labels and title
ax000.set_xlabel("RetractionIndexingAgreement_Score(%)")
ax000.set_ylabel("Fraction of IDs in Unionlist(%)")
# ax3_01_1.set_title("Percentage of DOIs with their RIA_Score in Unionlist", size=13)

fig000.patch.set_facecolor('#ccd9ff') 
plt.tight_layout(pad=2.0)

In [ ]:
"""
Distribution of RetractionIndexingAgreement of the new Unionlist_CRWS (exclusive source filtered)
"""
newly_added_crws_filtered = unionlist_crws_filtered[unionlist_crws_filtered['DOI'].isin(newly_added_crws['DOI'])]

table001= count_DOIs_for_RIA(newly_added_crws_filtered)
table001

In [ ]:
"""
Distribution of RetractionIndexingAgreement(RIA) of the STI2023 (exclusive source filtered)
- We obtain STI2023 (excluding exclusive indexed & covered items) from Unionlist_CRWS -- using present RIA 
"""
sti2023_filtered = unionlist_crws_filtered[unionlist_crws_filtered['DOI'].isin(unionlist_sti['DOI'])]

table002= count_DOIs_for_RIA(sti2023_filtered)
table002

In [ ]:
"""
Retraction Indexing Agreement by Items (in Group) for the newly added DOIs(from Unionlist_CRWS) 
to STI2023 unionlist
"""

plt.style.use('bmh')
fig001, ax001 = plt.subplots(figsize=(14, 8))

mycolor_001 = dict(zip(table001['RIA_Score(%)'], table001['Color']))

# Create the bar plot
sns.barplot(y=table001['Fraction_in_Unionlist(%)'], x=table001['RIA_Score(%)'],
            ax=ax001, color="#1f77b4",palette=mycolor_001, ci=None)

#Add custom legend
legend_labels = {
    'red': '<=25%',
    'orange': '>25% to 50%',
    '#1f77b4': '>50% to 75%',
    'green': '100%',

}

legend_handles = [mpatches.Patch(color=color, label=label) for color, label in legend_labels.items()]

ax001.legend(handles=legend_handles, title="RIA Score Range", loc="upper left")


# Label bars with noDOIs values
for bar, noDOIs, frc in zip(ax001.patches, table001['noDOIs'],table001['Fraction_in_Unionlist(%)']):
    ax001.annotate(f"{noDOIs} Items ({frc}%)", #f"{noDOIs} / {len(unionlist3)}"
                      xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
                      xytext=(0, 3),  # 3 points vertical offset
                      textcoords="offset points",
                      ha='center', va='bottom')

# Set labels and title
ax001.set_xlabel("RetractionIndexingAgreement_Score(%)")
ax001.set_ylabel("Fraction of IDs in Unionlist(%)")
# ax3_01_1.set_title("Percentage of DOIs with their RIA_Score in Unionlist", size=13)

fig001.patch.set_facecolor('#ccd9ff') 
plt.tight_layout(pad=2.0)

In [ ]:
"""
Retraction Indexing Agreement by Items (in Group) for STI2023
"""

plt.style.use('bmh')
fig002, ax002 = plt.subplots(figsize=(14, 8))

mycolor_002 = dict(zip(table002['RIA_Score(%)'], table002['Color']))

# Create the bar plot
sns.barplot(y=table002['Fraction_in_Unionlist(%)'], x=table002['RIA_Score(%)'],
            ax=ax002, color="#1f77b4",palette=mycolor_002, ci=None)

#Add custom legend
#Add custom legend
legend_labels = {
    'red': '<=25%',
    'orange': '>25% to 50%',
    '#1f77b4': '>50% to 75%',
    'green': '100%',
}

legend_handles = [mpatches.Patch(color=color, label=label) for color, label in legend_labels.items()]

ax002.legend(handles=legend_handles, title="RIA Score Range", loc="upper left")


# Label bars with noDOIs values
for bar, noDOIs, frc in zip(ax002.patches, table002['noDOIs'],table002['Fraction_in_Unionlist(%)']):
    ax002.annotate(f"{noDOIs} IDs ({frc}%)", #f"{noDOIs} / {len(unionlist3)}"
                      xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
                      xytext=(0, 3), # 3 points vertical offset
                      textcoords="offset points",
                      ha='center', va='bottom')

# Set labels and title
ax002.set_xlabel("RetractionIndexingAgreement_Score(%)")
ax002.set_ylabel("Fraction of IDs in Unionlist(%)")
# ax3_01_1.set_title("Percentage of DOIs with their RIA_Score in Unionlist", size=13)

fig002.patch.set_facecolor('#ccd9ff') 
plt.tight_layout(pad=2.0)

In [ ]:
"""
Subplot for table000 & table0001
"""

plt.style.use('bmh')
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))

# Plotting table002 -  STI2023
mycolor_002 = dict(zip(table000['RIA_Score(%)'], table000['Color']))
sns.barplot(y=table002['Fraction_in_Unionlist(%)'], x=table000['RIA_Score(%)'], 
            ax=ax1, palette=mycolor_000, ci=None)
ax1.set_title('April 2023 union list',fontsize=15)
ax1.set_xlabel("Retraction Indexing Agreement Score (%)")
ax1.set_ylabel("Fraction of IDs in Unionlist (%)")
ax1.tick_params(axis='y', labelsize=14) 



# Plotting table001 - Newly added items to STI2023
mycolor_001 = dict(zip(table001['RIA_Score(%)'], table001['Color']))
sns.barplot(y=table001['Fraction_in_Unionlist(%)'], x=table001['RIA_Score(%)'], 
            ax=ax2, palette=mycolor_001, ci=None)
ax2.set_title('Items added to the union list: May 2023 to July 2024',fontsize=15)
ax2.set_xlabel("Retraction Indexing Agreement Score (%)", fontsize=12)
ax2.set_ylabel("")
# ax2.yaxis.set_visible(True)
ax2.tick_params(axis='x', labelsize=14) 
ax2.set_xticks([])

# Remove y-axis labels and ticks from the second subplot
ax2.set_yticklabels([])
ax2.set_yticks([])

# Find the maximum y-value for setting the same y-axis scale
max_y = max(table000['Fraction_in_Unionlist(%)'].max(), table001['Fraction_in_Unionlist(%)'].max())

# Set the same y-axis limits and ticks for both subplots
for ax in [ax1, ax2]:
    ax.set_ylim(0, max_y + 10)  # Add some padding at the top
    ax.set_yticks(range(0, int(max_y) + 11, 10))
    ax.set_xticks([])



# Create a combined legend
legend_labels = {
    'red': '<=25%',
    'orange': '>25% to 50%',
    '#1f77b4': '>50% to 75%',
    'green': '100%'
}

legend_handles = [mpatches.Patch(color=color, label=label) for color, label in legend_labels.items()]

# Place the combined legend outside the top of the plot
fig.legend(handles=legend_handles, title="Retraction Indexing Agreement Score Range", title_fontsize=15,\
           loc="upper center", bbox_to_anchor=(0.5, 1.10), ncol=4,prop={'size': 15})

# Label bars with noDOIs values for table000
for bar, noDOIs, frc in zip(ax1.patches, table002['noDOIs'], table002['Fraction_in_Unionlist(%)']):
    ax1.annotate(f"{noDOIs} Items ({frc}%)",
                  xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
                  xytext=(0, 3),
                  textcoords="offset points",fontsize=12.5, 
                  ha='center', va='bottom')

# Label bars with noDOIs values for table001
for bar, noDOIs, frc in zip(ax2.patches, table001['noDOIs'], table001['Fraction_in_Unionlist(%)']):
    ax2.annotate(f"{noDOIs} Items ({frc}%)",
                  xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
                  xytext=(0, 3),
                  textcoords="offset points",fontsize=12.5, 
                  ha='center', va='bottom')

# Customize figure appearance
fig.patch.set_facecolor('#ccd9ff')
plt.tight_layout(pad=1.0, rect=[0, 0, 1, 1])  # Adjust layout to make room for legend

# plt.savefig(result_dir+'unionlist_ria_proportion_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

### Investigating Missing DOIs

In [ ]:
print(f'The Actual # DOIs in STI2023 found in unionlist_crws 2024 is {len(common_df)}')

missing_sti_dois = set(unionlist_sti['DOI']) - set(common_df['DOI'])
print(f'The # missing DOIs of STI2023 not found in unionlist_crws 2024 is {len(missing_sti_dois)}')

In [ ]:
"""
Data Quality Check:
Missing records in STI2023 when cross-matched with unionlist_crws 2024
"""

missing_df= unionlist_sti[unionlist_sti['DOI'].isin(missing_sti_dois)]

# missing_df.to_csv(result_dir+'missing_doi_insti2023.csv',index=False)
missing_df

In [ ]:
fig00, ax00 = plt.subplots(figsize=(15, 6))

# Group and count DOIs by year
missing_t = missing_df.groupby('Year')['DOI'].count().reset_index()

# Plot the lineplot
sns.lineplot(data=missing_t, x='Year', y='DOI', marker='o', ax=ax00, color='black', linestyle='--')

# Shade the area under the line plot
ax00.fill_between(missing_t['Year'], missing_t['DOI'], color='darkgray', alpha=1, label = 'missing DOI')

# Set the x-axis ticks to show every year
ax00.set_xticks(missing_t['Year'])
ax00.set_xticklabels(missing_t['Year'], rotation=45)

# Add titles and labels
ax00.set_title('Publication year distribution of numbers of missing DOIs in STI2023 union list ')
ax00.set_xlabel('Year')
ax00.set_ylabel('#DOI')
ax00.legend()

plt.show()

In [ ]:
# Group by year and count DOIs
sti_year_t = unionlist_sti.groupby('Year')['DOI'].count().reset_index()
ul_year_t = common_df.groupby('Year')['DOI'].count().reset_index()

ul_year_t= ul_year_t.iloc[:-1,:].copy() # Remove 2024 year because of plotting

In [ ]:
"""
Plotting trends of retracted publications in STI2023 indicating missing DOIs - 1
"""

fig01, ax01 = plt.subplots(figsize=(35, 20))

# Plot the line plots
sns.lineplot(data=sti_year_t, x='Year', y='DOI', marker='o', ax=ax01, color='black', label='STI Year')
sns.lineplot(data=ul_year_t, x='Year', y='DOI', marker='o', ax=ax01, color='grey', label='UL Year')

# Ensure x-axis is the same for both plots
ax01.set_xlim(min(sti_year_t['Year'].min(), ul_year_t['Year'].min()), max(sti_year_t['Year'].max(), ul_year_t['Year'].max()))

# Fill the area between the lines
# Ensure both DataFrames cover the same years
combined_years = sorted(set(sti_year_t['Year']).union(ul_year_t['Year']))
sti_year_t = sti_year_t.set_index('Year').reindex(combined_years, fill_value=0).reset_index()
ul_year_t = ul_year_t.set_index('Year').reindex(combined_years, fill_value=0).reset_index()

plt.fill_between(sti_year_t['Year'], sti_year_t['DOI'], ul_year_t['DOI'], where=(sti_year_t['DOI'] > ul_year_t['DOI']),
                 color='red', alpha=1, label='Missing DOIs in STI2023)')

# Set the x-axis ticks to show every year
ax01.set_xticks(combined_years)
ax01.set_xticklabels(combined_years, rotation=45)

# Increase font size of tick labels
ax01.tick_params(axis='both', which='major', labelsize=18)

# Add titles and labels
ax01.set_title('Number of DOIs Over Years',fontsize=16)
ax01.set_xlabel('Year',  fontsize=16)
ax01.set_ylabel('#DOI')
ax01.legend()

# Display the plot
plt.show()


In [ ]:
"""
Plotting trends of retracted publications in STI2023 indicating missing DOIs - 2
"""

fig01, ax01 = plt.subplots(figsize=(35, 20))
# Plot the line plots
sns.lineplot(data=sti_year_t, x='Year', y='DOI', marker='o', ax=ax01, color='black', label='STI Year')
sns.lineplot(data=ul_year_t, x='Year', y='DOI', marker='o', ax=ax01, color='grey', label='UL Year')

# Ensure x-axis is the same for both plots
ax01.set_xlim(min(sti_year_t['Year'].min(), ul_year_t['Year'].min()), max(sti_year_t['Year'].max(), ul_year_t['Year'].max()))

# Fill the area between the lines
# Ensure both DataFrames cover the same years
combined_years = sorted(set(sti_year_t['Year']).union(ul_year_t['Year']))
sti_year_t = sti_year_t.set_index('Year').reindex(combined_years, fill_value=0).reset_index()
ul_year_t = ul_year_t.set_index('Year').reindex(combined_years, fill_value=0).reset_index()

plt.fill_between(sti_year_t['Year'], sti_year_t['DOI'], ul_year_t['DOI'], where=(sti_year_t['DOI'] > ul_year_t['DOI']),
                 color='darkred', alpha=1, label='Missing DOIs in STI2023)')

plt.fill_between(sti_year_t['Year'], sti_year_t['DOI'], ul_year_t['DOI'], where=(sti_year_t['DOI'] < ul_year_t['DOI']),
                 color='darkred', alpha=1, )
# Set the x-axis ticks to show every year
ax01.set_xticks(combined_years)
ax01.set_xticklabels(combined_years, rotation=45)

# Increase font size of tick labels
ax01.tick_params(axis='both', which='major', labelsize=18)

# Add titles and labels
ax01.set_title('Number of DOIs Over Years',fontsize=16)
ax01.set_xlabel('Year',  fontsize=16)
ax01.set_ylabel('#DOI')
ax01.legend()


###
fig00, ax00 
ax00= fig01.add_axes([0.2, 0.2, 0.38, 0.6],  frameon=True,)  #[0.18, 0.5, 0.55, 0.35] [left, bottom, width, height]
for spine in ax00.spines.values():
    spine.set_edgecolor('black')
    
# Plot the lineplot
sns.lineplot(data=missing_t, x='Year', y='DOI', marker='o', ax=ax00, color='black', linestyle='--')

# Shade the area under the line plot
ax00.fill_between(missing_t['Year'], missing_t['DOI'], color='lightcoral', alpha=1, label = 'missing DOI')

# color='lightcoral' - light red
# Set the x-axis ticks to show every year
ax00.set_xticks(missing_t['Year'])
ax00.set_xticklabels(missing_t['Year'], rotation=45)

# Add titles and labels
ax00.set_title('Publication year distribution of numbers of missing DOIs in STI2023 union list ')
ax00.set_xlabel('Year')
ax00.set_ylabel('#DOI')
ax00.legend()

# Display the plot
plt.show()


In [ ]:
"""
Examining changes between STI2023 and the unionlist_CRWS2024 unionlists
"""

combined_df= pd.merge(sti_year_t,ul_year_t, how='inner',on='Year')

combined_df.rename(columns={'DOI_x': 'DOI_STI', 'DOI_y':'DOI_Present'}, inplace=True)
combined_df

In [ ]:
"""
Calculating the breakdown of missing DOIs in STI2023 union list

Extract missing DOIs for each sources
"""

# missing_df[missing_df['source_old'].apply(lambda col: col if 'WoS_Core' in col)]
msDOI_wos= missing_df[missing_df['source'].str.contains('Web of Science')]['DOI'].tolist()
msDOI_crossref= missing_df[missing_df['source'].str.contains('Crossref')]['DOI'].tolist()
msDOI_scopus= missing_df[missing_df['source'].str.contains('Scopus')]['DOI'].tolist()
msDOI_rw= missing_df[missing_df['source'].str.contains('Retraction Watch')]['DOI'].tolist()

print(f'# total missing DOIs in STI2023 is {len(missing_df)} DOIs')
print(f'# total missing DOIs in Crossref is {len(msDOI_crossref)} DOIs')
print(f'# total missing DOIs in Retraction Watch is {len(msDOI_rw)} DOIs')
print(f'# total missing DOIs in Scopus is {len(msDOI_scopus)} DOIs')
print(f'# total missing DOIs in Web of Science is {len(msDOI_wos)} DOIs')

In [ ]:
msDOI_wos

In [ ]:
"""
Write missing DOIs in STI2023 in files 
"""

# Missing DOIs in Crossref
with open(result_dir+"sti_missingdoi_incrossref.txt", "w") as file:
    for item in msDOI_crossref:
        file.write(item + "\n")

# Missing DOIs in Retraction Watch
with open(result_dir+"sti_missingdoi_inretractionwatch.txt", "w") as file:
    for item in msDOI_rw:
        file.write(item + "\n")
        
# Missing DOIs in Scopus
with open(result_dir+"sti_missingdoi_inscopus.txt", "w") as file:
    for item in msDOI_scopus:
        file.write(item + "\n")
        
# Missing DOIs in Web of Science
with open(result_dir+"sti_missingdoi_inwos.txt", "w") as file:
    for item in msDOI_wos:
        file.write(item + "\n")

In [ ]:
def batch_items(pmids:list, cut:int)-> list[list]:
    """
    It divides the list of items into batches for processing. 
    :param pmids: list of items 
    :param cut: maximum number of records to assign to a batch
    
    :return: list of list of batches of pmids
    """
    pmids_batches=[]
    
    while len(pmids) >= cut:
        selected_pmids= pmids[:cut]
        pmids_batches.append(selected_pmids)
#         print(selected_pmids)    
        pmids = pmids[cut:]

    if pmids:
        pmids_batches.append(pmids)
#         print(pmids)

    return pmids_batches



#### Checking missing DOIs coverage in Crossref


In [ ]:
from crossref.restful import Works, Etiquette
my_etiquette = Etiquette('Retraction Indexing Assessment', 'version2', 'no url', my_email)
#my_etiquette = Etiquette('My Project Name', 'My Project version', 'My Project URL', 'My contact email')
works = Works(etiquette=my_etiquette)


In [ ]:
cut= 50
check_doi_in_crossref= msDOI_crossref 

check_doi_in_crossref_batches = batch_items(check_doi_in_crossref, cut)

print(f'The total items to search in Crossref is {len(check_doi_in_crossref)}, which are divided into {len(check_doi_in_crossref_batches)} batches')

In [ ]:
"""
Check DOI Coverage in Crossref
"""

new = Works()
start = time.time()

crossref_covered_doi=[]

for i in tqdm(check_doi_in_crossref):
    try:
        for j in new.filter(doi = i).select('DOI'):
            find = j['DOI']
            if i == find:
                crossref_covered_doi.append(i)

    except Exception:
            pass
    time.sleep(0.10)
        #print(i)


In [ ]:
print(f'The total # missing DOIs found in Crossref is {len(crossref_covered_doi)}')
print (f'The missing DOIs now not covered in Crossref is {len(set(msDOI_crossref) - set(crossref_covered_doi))} DOI')
set(msDOI_crossref) - set(crossref_covered_doi)

In [ ]:
"""
All the missing DOIs in Crossef were found - # already written to file -> sti_missingdoi_incrossref
"""

In [ ]:
"""
Investigate found missing DOIs in Crossref
"""

crossref_covered_doi

In [ ]:
# DOIs that starts with '10.1016/j.ijdevneu.2015.' ... '10.1016/j.ijdevneu.2015.04.317'
c = 0
for doi in crossref_covered_doi:
    if doi.startswith('10.1016/j.ijdevneu.2015.'):
        c+=1 
print(f"The total # DOI that starts with '10.1016/j.ijdevneu.2015.04....': {c}")

#### Checking missing DOIs coverage in Scopus

In [ ]:
"""
Break the list of DOIs to search into batches of a maximum of 25 items in each batch. 
The limit per page for Scopus is 25 items at a time.

Input:
    cut: maximum number items in a batch
"""
scopus_cut= 25
check_doi_in_scopus= msDOI_scopus 

check_doi_in_scopus_batches = batch_items(check_doi_in_scopus, scopus_cut) 


print(f'The total items to search in Scopus is {len(check_doi_in_scopus)}, which are divided into {len(check_doi_in_scopus_batches)} batches')
print(f'The items list is divided into lists in which each list contains {scopus_cut} records maximum')

In [ ]:
"""
Checking DOIs coverage in Scopus
"""

scopus_covered_doi = []
scopus_unresolved_doi = []

# Set your API key
api_key = elsevier_api_key
elsevier_insttoken = elsevier_insttoken

url_base = "https://api.elsevier.com/content/search/scopus" 
headers = {
        'X-ELS-APIKey': api_key,
        'Accept': 'application/json',
        'X-ELS-Insttoken':elsevier_insttoken}


for batch in tqdm(check_doi_in_scopus_batches[:]):
    
    # Putting search format 'DOI(10.xxxx/xxxxxxxxxxxx) OR DOI(...)'
    check_now = ' OR '.join(f'DOI({doi})' for doi in batch) 
    params= {"query": check_now}
        
    response = requests.get(url_base,
                        headers= headers,
                    params = params)
    
    if response.status_code == 200:
        results = response.json()

        store=[]
            
        totalresult= int(results['search-results'].get('opensearch:totalResults',0))
    
        if totalresult > 0:
        
            try:
                for result in results['search-results']['entry']:
                    try:
                        store.append(result['prism:doi'])
                    except KeyError:
                        pass
                        scopus_unresolved_doi.extend(batch)
            except KeyError:
                pass
    else:
        scopus_unresolved_doi.extend(batch)

    scopus_covered_doi.extend(store)
    
    time.sleep(0.15)


In [ ]:
print(f'The total # missing DOIs covered in Scopus is {len(scopus_covered_doi)}')
print (f'The missing DOIs now not covered in Scopus is {len(set(msDOI_scopus) - set(scopus_covered_doi))} DOI')
set(msDOI_scopus) - set(scopus_covered_doi)
# scopus_unresolved_doi

In [ ]:
"""
All the missing DOIs in Scopus were found - # already written to file -> sti_missingdoi_inscopus
except {'10.1063/1.5032937'}
"""
scopus_covered_doi

#### Checking missing DOIs coverage in Web of Science

In [ ]:
def wos_check_DOI(DOIs_lists: list, database:str):
    """
    It checks the coverage of DOI in the Web of Science
    
    :param DOIs_lists: list of list of DOIs preformatted to query form for Web of Science use
    :param database: specific the database to search
    
    :return: list of list [available DOIs, DOIs that ran into error]
    
    """
    c=0
    wos_covered_doi = []
    wos_unresolved_doi = []

    # Define your API key
    WoS_api_key = wos_api_key


    # Set the headers with the API key
    headers = {
            'X-ApiKey': WoS_api_key,
            'charset': 'UTF-8',
            'Encoding': 'UTF-8',
            'content-type':'text/xml'
            }
#     base_url = 'https://api.clarivate.com/apis/wos-starter/v1/documents' 
    url = 'https://api.clarivate.com/apis/wos-starter/v1/documents'

    for batch in tqdm(DOIs_lists):
        check_now = ' OR '.join(f'DO=({doi})' for doi in batch) 
#         print(check_now)

        params = {
            'db': database, #'WOK',#'WOS',
            'q': check_now,
            'limit': 50
        }



        # Make the API request
        response = requests.get(url, params=params, headers=headers)

        # Check if the request was successful (200 status code)
        if response.status_code == 200:
#             print('success')
            # Extract the response content as JSON
            data = response.json()
            # Print the DOI details
            try:
                dois_result = data['hits']
                for doi in dois_result:
                    try:
                        if doi.get('identifiers'):
                            identifiers = doi.get('identifiers','')
                            DOI= identifiers.get('doi','')
                            wos_covered_doi.append(DOI)
                    except Exception:
                        pass
            except Exception:
                pass
                wos_unresolved_doi.append(check_now)

        else:
            # If the request was not successful, print the error message
            print(f"Request failed with status code: {response.status_code} in batch: {c}")
            #print(response.text)

        time.sleep(0.15)
        #print('batch: ',c)
        c+=1 
        
    return [wos_covered_doi,wos_unresolved_doi]


In [ ]:
"""
Break the list of DOIs to search into batches of a maximum of 50 items in each batch. 
The limit per page for Web of Science is 50 items at a time.

Input:
    cut: maximum number items in a batch
"""
wos_cut= 50
check_doi_in_woscore = msDOI_wos #woscoreDOI_notindexed['DOI'].tolist()# change doilist_wos to notindexedasretracted_source

check_doi_in_woscore_batches = batch_items(check_doi_in_woscore, wos_cut)

print(f'The total items to search in Web of Science Core is {len(check_doi_in_woscore)}, which are divided into {len(check_doi_in_woscore_batches)} batches')
print(f'The items list is divided into lists in which each list contains {wos_cut} records maximum')

In [ ]:
"""
Check DOIs Coverage in Web of Science Core
"""
start = time.time()

woscore_results= wos_check_DOI(check_doi_in_woscore_batches, 'WOS')

end = time.time()
end - start

In [ ]:
# Extracting DOIs that are still missing
notfound_doi_wos= list(set(msDOI_wos) - set(woscore_results[0]))

print(f'The total # missing DOIs found in Web of Science is {len(woscore_results[0])}')
print (f'The missing DOIs now not covered in Web of Science is {len(notfound_doi_wos)} DOI')
notfound_doi_wos[:15]

In [ ]:
woscore_results[0][:15]

In [ ]:
# Missing DOIs that were found in Web of Science
with open(result_dir+"sti_missingdoi_inwos_found.txt", "w") as file:
    for item in woscore_results[0]:
        file.write(item + "\n")
        

# Missing DOIs that were not found in Web of Science
with open(result_dir+"sti_missingdoi_inwos_notfound.txt", "w") as file:
    for item in notfound_doi_wos:
        file.write(item + "\n")

#### Checking missing DOIs coverage in Retraction Watch

In [ ]:
"""
Load Retraction Watch - downloaded on Aug 20, 2024
"""

rw_df= pd.read_csv(data_dir+'/retractionwatch/retractionwatch_aug20_2024.csv')
rw_df

In [ ]:
rw_df['OriginalPaperDOI']= rw_df['OriginalPaperDOI'].str.lower()
rw_df['RetractionDOI'] = rw_df['RetractionDOI'].str.lower()

In [ ]:
# Check missing DOIs in Retraction Watch Original DOI
rw_df[rw_df['OriginalPaperDOI'].isin(msDOI_rw)].count()[0]

In [ ]:
# Check missing DOIs in Retraction Watch RetractionDOI  (i.e. Retraction Notices)
rw_df[rw_df['RetractionDOI'].isin(msDOI_rw)].count()[0]

In [ ]:
# Printing  missing DOIs covered in Retraction Watch
missingdoi_foundin_rw= rw_df[rw_df['RetractionDOI'].isin(msDOI_rw)]['RetractionDOI'].tolist()
missingdoi_foundin_rw[:15]

In [ ]:
# Printing  missing DOIs not covered in Retraction Watch
missingdoi_notfoundin_rw = list(set(msDOI_rw) - set(missingdoi_foundin_rw))
missingdoi_notfoundin_rw[:15] 

In [ ]:
# Missing DOIs that were found in Retraction Watch
with open(result_dir+"sti_missingdoi_inrw_found.txt", "w") as file:
    for item in missingdoi_foundin_rw:
        file.write(item + "\n")
        

# Missing DOIs that were not found in Retraction Watch
with open(result_dir+"sti_missingdoi_inrwatch_notfound.txt", "w") as file:
    for item in missingdoi_notfoundin_rw :
        file.write(item + "\n")

#### Investigate any overlap in missing DOIs and RQ4 DOIs

In [ ]:
rq4_df = pd.read_excel(retraction_index_path+'/Spreadsheet for RQ4_main category.xlsx')


In [ ]:
rq4_missing_dois= missing_df[missing_df['DOI'].isin(rq4_df['DOI'].str.lower())]['DOI'].tolist()
rq4_missing_dois

In [ ]:
rq4_df[rq4_df['DOI'].isin(rq4_missing_dois)]